In [1]:
!pip3 -q install datasets transformers evaluate huggingface_hub bitsandbytes

In [2]:
!pip3 -q install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.5 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")
dataset_dict

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [21]:
device = torch.device('cuda')
model_path = "majorSeaweed/Bert_Uncased_FishingURL_Teacher"

tokenizer = AutoTokenizer.from_pretrained(model_path)
teacher_model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [22]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig

config = DistilBertConfig(n_heads=8, n_layers=4)
student_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                                    config=config,).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [24]:
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return accuracy, precision, recall, f1

In [25]:
def distillation_loss(student_logits, teacher_logits, true_labels, temperature, alpha):
    soft_targets = nn.functional.softmax(teacher_logits / temperature, dim=1)
    student_soft = nn.functional.log_softmax(student_logits / temperature, dim=1)

    distill_loss = nn.functional.kl_div(student_soft, soft_targets, reduction='batchmean') * (temperature ** 2)
    hard_loss = nn.CrossEntropyLoss()(student_logits, true_labels)

    loss = alpha * distill_loss + (1.0 - alpha) * hard_loss

    return loss

In [26]:
batch_size = 32
lr = 1e-4
num_epochs = 10
temperature = 2.0
alpha = 0.5

In [29]:
optimizer = optim.Adam(student_model.parameters(), lr=lr)
dataloader = DataLoader(tokenized_dataset['train'], batch_size=batch_size)
test_dataloader = DataLoader(tokenized_dataset['test'], batch_size=batch_size)
val_dataloader = DataLoader(tokenized_dataset['validation'], batch_size=batch_size)

In [30]:
student_model.train()

for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_outputs.logits

        student_outputs = student_model(input_ids, attention_mask=attention_mask)
        student_logits = student_outputs.logits

        loss = distillation_loss(student_logits, teacher_logits, labels, temperature, alpha)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed with loss: {loss.item()}")

    teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, val_dataloader, device)
    print(f"Teacher (validation) - Accuracy: {teacher_accuracy:.4f}, Precision: {teacher_precision:.4f}, Recall: {teacher_recall:.4f}, F1 Score: {teacher_f1:.4f}")

    student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, val_dataloader, device)
    print(f"Student (validation)) - Accuracy: {student_accuracy:.4f}, Precision: {student_precision:.4f}, Recall: {student_recall:.4f}, F1 Score: {student_f1:.4f}")
    print("\n")

    student_model.train()

Epoch 1 completed with loss: 0.14367979764938354
Teacher (validation) - Accuracy: 0.8711, Precision: 0.9073, Recall: 0.8267, F1 Score: 0.8651
Student (validation)) - Accuracy: 0.8867, Precision: 0.9485, Recall: 0.8178, F1 Score: 0.8783


Epoch 2 completed with loss: 0.10114365816116333
Teacher (validation) - Accuracy: 0.8711, Precision: 0.9073, Recall: 0.8267, F1 Score: 0.8651
Student (validation)) - Accuracy: 0.9311, Precision: 0.9330, Recall: 0.9289, F1 Score: 0.9310


Epoch 3 completed with loss: 0.11216779053211212
Teacher (validation) - Accuracy: 0.8711, Precision: 0.9073, Recall: 0.8267, F1 Score: 0.8651
Student (validation)) - Accuracy: 0.9356, Precision: 0.9495, Recall: 0.9200, F1 Score: 0.9345


Epoch 4 completed with loss: 0.11616306006908417
Teacher (validation) - Accuracy: 0.8711, Precision: 0.9073, Recall: 0.8267, F1 Score: 0.8651
Student (validation)) - Accuracy: 0.8733, Precision: 0.9884, Recall: 0.7556, F1 Score: 0.8564


Epoch 5 completed with loss: 0.07720547914505005

In [31]:
teacher_accuracy, teacher_precision, teacher_recall, teacher_f1 = evaluate_model(teacher_model, val_dataloader, device)
print(f"Teacher (test) - Accuracy: {teacher_accuracy:.4f}, Precision: {teacher_precision:.4f}, Recall: {teacher_recall:.4f}, F1 Score: {teacher_f1:.4f}")
student_accuracy, student_precision, student_recall, student_f1 = evaluate_model(student_model, val_dataloader, device)
print(f"Student (test) - Accuracy: {student_accuracy:.4f}, Precision: {student_precision:.4f}, Recall: {student_recall:.4f}, F1 Score: {student_f1:.4f}")

Teacher (test) - Accuracy: 0.8711, Precision: 0.9073, Recall: 0.8267, F1 Score: 0.8651
Student (test) - Accuracy: 0.9267, Precision: 0.9486, Recall: 0.9022, F1 Score: 0.9248


In [32]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
student_model.push_to_hub('majorSeaweed/Bert_Uncased_FishingURL_distilled')
tokenizer.push_to_hub('majorSeaweed/Bert_Uncased_FishingURL_distilled')

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/211M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/majorSeaweed/Bert_Uncased_FishingURL_distilled/commit/613ae53620bd6ed41bd32cbcb0c460ddeae7055a', commit_message='Upload tokenizer', commit_description='', oid='613ae53620bd6ed41bd32cbcb0c460ddeae7055a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/majorSeaweed/Bert_Uncased_FishingURL_distilled', endpoint='https://huggingface.co', repo_type='model', repo_id='majorSeaweed/Bert_Uncased_FishingURL_distilled'), pr_revision=None, pr_num=None)